In [2]:
import sys
#sys.path.append('./Tiny_Faces_in_Tensorflow/')
#import tiny_face_eval as tiny
import evaluate
from metrics import *
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sklearn.metrics import mean_squared_error as mse
import glob
import os
import cv2
import pandas as pd
from sklearn.svm import SVC
import numpy as np
import imp
import time
import random
import detect
#import dlib
from imgaug import augmenters as iaa
#imp.reload(tiny)
#imp.reload(detect)
%matplotlib inline
from IPython import embed
from tqdm import tqdm

In [2]:
#weights_path = './Tiny_Faces_in_Tensorflow/hr_res101.pkl'
weights_path = '/home/paula/THINKSMARTER_/face-detectors/Tiny_Faces/hr_res101.pkl'

## Saving frames

In [3]:
#cap = cv2.VideoCapture('/home/alexattia/Work/RecVis/famvk.avi')
#cap = cv2.VideoCapture('/home/paula/THINKSMARTER_/videoplayback.mp4')
cap = cv2.VideoCapture('test.avi')

fps = cap.get(cv2.CAP_PROP_FPS)
#initial_target = int(45 * fps) + 10
#final_target = int(49 * fps) + 10
i = 0
frames = []
while(True):
    ret, frame = cap.read()
    if ret:
        frames.append(frame[:,:,::-1])
    else:
        break
    #i +=1 
    #if i in range(initial_target, final_target+10):
        #frames.append(frame[:,:,::-1])
    #if i == final_target:
        #break

In [4]:
images = []
# LEN -2 IF IT CRASHES
for k in range(0, len(frames), 10):
    try:
        imgs = [frames[k], frames[k+1], frames[k+2], frames[k+10]]
    except IndexError:
        imgs = [frames[k], frames[k+1], frames[k+2], frames[len(frames)-1]]
    images.append(imgs)

In [5]:
len(frames)

260

In [6]:
len(images)

26

## Detection

In [16]:
all_detections = []
for frames in images:
    detections = []
    for frame in tqdm(frames):
        with tf.Graph().as_default():
            b = evaluate.evaluate(weight_file_path=weights_path,  img=frame)
        detections.append(b)
    all_detections.append(detections)

  0%|          | 0/4 [00:00<?, ?it/s]/home/paula/THINKSMARTER_/Model/ExtendedTinyFaces/evaluate.py:111: MatplotlibDeprecationWarning: numpy.arange
  scales_down = pl.frange(min_scale, 0, 1.)
/home/paula/THINKSMARTER_/Model/ExtendedTinyFaces/evaluate.py:112: MatplotlibDeprecationWarning: numpy.arange
  scales_up = pl.frange(0.5, max_scale, 0.5)
100%|██████████| 4/4 [02:05<00:00, 31.46s/it]


In [7]:
all_detections = np.load('numpy_alldetections.npy')


In [8]:
print(len(all_detections))

len(frames)

26


260

In [9]:
images[0][0].shape #--> 26 x 4 x (1080, 1920, 3)

(1080, 1920, 3)

In [10]:
frames[0].shape  # 260 x (1080, 1920, 3)

(1080, 1920, 3)

In [11]:

all_detections.shape    # --> (26, 4)
all_detections[0][0].shape # --> 4 x (156, 4)



(156, 4)

## Matching 

In [26]:
threshold = 0.55
matcheds = []
t0 = time.time()
for j in range(len(images)):
    frames = images[j]
    detections = all_detections[j]
    matched = 0
    t0bis = time.time()
    for p in range(len(detections[0])):
        neigh_detect, distances = detect.train_binclas(frames, detections, p)
    
        idx_max, val_max = np.argmax(distances[:,1]), np.max(distances[:,1])
        if val_max > threshold:
            matched += 1
    matcheds.append(matched)
    t1 = time.time()
    print('It took %.1f sec i.e %.2f/detection' % (t1-t0bis, (t1-t0bis)/len(detections[0])))
print('Total : %.1f' % (time.time() - t0))

It took 145.5 sec i.e 0.93/detection
It took 118.7 sec i.e 0.63/detection
It took 89.9 sec i.e 0.58/detection
It took 91.3 sec i.e 0.63/detection
It took 89.3 sec i.e 0.55/detection
It took 75.7 sec i.e 0.54/detection
It took 82.1 sec i.e 0.60/detection
It took 98.2 sec i.e 0.60/detection
It took 105.6 sec i.e 0.68/detection
It took 108.2 sec i.e 0.60/detection
It took 78.8 sec i.e 0.56/detection
It took 77.8 sec i.e 0.58/detection
It took 87.4 sec i.e 0.62/detection
It took 103.8 sec i.e 0.67/detection
It took 119.9 sec i.e 0.69/detection
It took 124.8 sec i.e 0.70/detection
It took 115.8 sec i.e 0.64/detection
It took 119.2 sec i.e 0.73/detection
It took 135.3 sec i.e 0.70/detection
It took 109.0 sec i.e 0.59/detection
It took 79.0 sec i.e 0.52/detection
It took 67.0 sec i.e 0.53/detection
It took 58.9 sec i.e 0.47/detection
It took 42.8 sec i.e 0.41/detection
It took 33.0 sec i.e 0.39/detection
It took 31.6 sec i.e 0.42/detection
Total : 2388.5


In [13]:
#np.save('matcheds',matcheds)
matcheds = np.load('matcheds.npy')


In [14]:
matcheds

array([156, 178, 153, 141, 159, 138, 132, 160, 153, 176, 139, 122, 137,
       148, 167, 174, 165, 159, 187, 182, 145, 116, 120, 103,  76,  67])

In [15]:
len(all_detections[1][0])

187

## Counting

In [16]:
s = 0

#for j in range(10):
for j in range(len(all_detections)):
    detections = all_detections[j]
    s += len(detections[0]) - matcheds[j]
s += len(detections[3])

In [17]:
s

222

## Gif Producting with counting

In [25]:
"""
#cap = cv2.VideoCapture('/home/alexattia/Work/RecVis/famvk.avi')
#cap = cv2.VideoCapture('/home/paula/THINKSMARTER_/10km_de_course.wmv.mp4')
#cap = cv2.VideoCapture('/home/paula/THINKSMARTER_/videoplayback.mp4')
cap = cv2.VideoCapture('test.avi')

fps = cap.get(cv2.CAP_PROP_FPS)
initial_target = int(45 * fps) + 10
final_target = int(49 * fps) + 10
i = 0
frames = []
while(True):
    ret, frame = cap.read()
    if ret:
        frames.append(frame[:,:,::-1])
    else:
        break
    #i +=1 
    #if i in range(initial_target, final_target, 1):
    #   frames.append(frame[:,:,::-1])
    #if i == final_target:
    #   break
"""

In [18]:
import sys
sys.path.append('/home/paula/THINKSMARTER_/Model/ExtendedTinyFaces/')

In [19]:
import evaluate as tiny_evaluate


In [20]:
from tqdm import tqdm

In [37]:
len(frames)

260

In [41]:
detections = []
detections_faces = []
for i, frame in enumerate(tqdm(frames)):
    with tf.Graph().as_default():
        b = tiny_evaluate.evaluate(weight_file_path=weights_path, data_dir='.jpg', output_dir='', img=frame,
                          prob_thresh=0.5, nms_thresh=0.1, lw=3, 
                          display=False, save=False, draw=False, print_=0)
    detections.append(b[0])
    detections_faces.append(b)
    time.sleep(0.5)




  0%|          | 0/260 [00:00<?, ?it/s]/home/paula/THINKSMARTER_/Model/ExtendedTinyFaces/evaluate.py:111: MatplotlibDeprecationWarning: numpy.arange
  scales_down = pl.frange(min_scale, 0, 1.)
/home/paula/THINKSMARTER_/Model/ExtendedTinyFaces/evaluate.py:112: MatplotlibDeprecationWarning: numpy.arange
  scales_up = pl.frange(0.5, max_scale, 0.5)



  0%|          | 1/260 [00:44<3:13:06, 44.74s/it]


  1%|          | 2/260 [01:28<3:11:03, 44.43s/it]


  1%|          | 3/260 [02:12<3:09:03, 44.14s/it]


  2%|▏         | 4/260 [02:56<3:07:58, 44.06s/it]


  2%|▏         | 5/260 [03:39<3:06:44, 43.94s/it]


  2%|▏         | 6/260 [04:22<3:05:32, 43.83s/it]


  3%|▎         | 7/260 [05:06<3:04:31, 43.76s/it]


  3%|▎         | 8/260 [05:49<3:03:43, 43.74s/it]


  3%|▎         | 9/260 [06:33<3:02:49, 43.70s/it]


  4%|▍         | 10/260 [07:17<3:02:09, 43.72s/it]


  4%|▍         | 11/260 [08:00<3:01:22, 43.70s/it]


  5%|▍         | 12/260 [08:44<3:00:41, 43.72s/it]


  5%|▌         | 13

 97%|█████████▋| 253/260 [2:34:38<04:16, 36.68s/it]


 98%|█████████▊| 254/260 [2:35:05<03:39, 36.64s/it]


 98%|█████████▊| 255/260 [2:35:32<03:02, 36.60s/it]


 98%|█████████▊| 256/260 [2:35:58<02:26, 36.56s/it]


 99%|█████████▉| 257/260 [2:36:25<01:49, 36.52s/it]


 99%|█████████▉| 258/260 [2:36:51<01:12, 36.48s/it]


100%|█████████▉| 259/260 [2:37:18<00:36, 36.44s/it]


100%|██████████| 260/260 [2:37:44<00:00, 36.40s/it]




In [42]:
#np.save('numpy_detections_0',detections)
#np.save('numpy_detections_justFaces',detections_faces)


In [21]:
detections = np.load('numpy_detections_0.npy')


In [52]:
detections_faces = np.load('numpy_detections_justFaces.npy')


In [58]:
len(detections_faces) # ..> 4

detections_faces[0].shape




(156, 4)

In [59]:
len(all_detections[0][3])

187

In [60]:
## Computing incremental count
nbs = []
init = len(all_detections[0][0])

#for j in range(1, 10):
for j in range(1, len(all_detections)):
    nbs.append(init)
    detections_ = all_detections[j]
    
    init += len(detections_[0]) - matcheds[j-1]
    
init += len(detections_[3]) - matcheds[j]
nbs.append(init)

In [61]:
nbs

[156,
 187,
 163,
 154,
 176,
 157,
 156,
 188,
 184,
 212,
 177,
 173,
 192,
 209,
 234,
 245,
 251,
 250,
 283,
 281,
 251,
 233,
 242,
 227,
 209,
 222]

In [66]:
from IPython import embed
k = 0
l = 0
images = []
ff = []
font = cv2.FONT_HERSHEY_SIMPLEX
frames[:-2]

for j, frame in enumerate(frames):
    img = frame.copy()
    
    #for detect_ in detections[j]:
    for detect_ in detections_faces[j]:
        pt1, pt2 = tuple(detect_[:2]), tuple(detect_[2:])
        cv2.rectangle(img, pt1, pt2, (255, 0, 0), 2)
    
    #pt1, pt2 = tuple(detections[j][:2]), tuple(detections[j][2:])
    #cv2.rectangle(img, pt1, pt2, (255, 0, 0), 2)
    #embed() 
    #cv2.putText(img, 'Incremental count : %d' % nbs[l], (1750,1300), font, 1.5, (0, 255, 0), 3)
    
    bottomLeftCornerOfText = (img.shape[1]-650,img.shape[0]-50)
    cv2.putText(img, 'Incremental count : %d' % nbs[l], bottomLeftCornerOfText , font, 1.5, (0, 255, 0), 3)
    #cv2.imwrite('frames_%05d.png' % j, img[:,:,::-1])
    
    #if j in range(10, 89, 9):
    if j in range(len(all_detections), 89, 9):
        l += 1
    images.append(img)   
    cv2.imwrite('./output_video_sample_all_faces/frames_%05d.png' % j, img[:,:,::-1])

In [71]:
for j, frame in enumerate(frames):

    if j in range(10, 89, 9):
        print(j)
  

10
19
28
37
46
55
64
73
82


# Creating a video from images

In [67]:

dir_path = './output_video_sample_all_faces'
listdir = os.listdir(dir_path)
listdir.sort()
images = []
for f in tqdm(listdir):
    if f.endswith('.png'):
        images.append(f)

image_path = os.path.join(dir_path, images[0])
frame = cv2.imread(image_path)
height, width, channels = frame.shape

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Be sure to use lower case
out = cv2.VideoWriter('test_ExtendedTinyFaces_allFaces.avi', fourcc, 20.0, (width, height))

for image in tqdm(images):

    image_path = os.path.join(dir_path, image)
    frame = cv2.imread(image_path)

    out.write(frame) # Write out frame to video
    if (cv2.waitKey(1) & 0xFF) == ord('q'): # Hit `q` to exit
        break

# Release everything if job is finished
out.release()
cv2.destroyAllWindows()




  0%|          | 0/260 [00:00<?, ?it/s]


100%|██████████| 260/260 [00:00<00:00, 276290.61it/s]


  0%|          | 0/260 [00:00<?, ?it/s]


  1%|          | 3/260 [00:00<00:13, 19.47it/s]


  2%|▏         | 5/260 [00:00<00:13, 18.25it/s]


  3%|▎         | 7/260 [00:00<00:13, 18.16it/s]


  3%|▎         | 9/260 [00:00<00:13, 18.12it/s]


  4%|▍         | 11/260 [00:00<00:14, 17.26it/s]


  5%|▌         | 13/260 [00:00<00:14, 17.30it/s]


  6%|▌         | 15/260 [00:00<00:14, 17.31it/s]


  7%|▋         | 17/260 [00:00<00:13, 17.40it/s]


  7%|▋         | 19/260 [00:01<00:13, 17.44it/s]


  8%|▊         | 21/260 [00:01<00:13, 17.52it/s]


  9%|▉         | 23/260 [00:01<00:13, 17.60it/s]


 10%|▉         | 25/260 [00:01<00:13, 17.70it/s]


 10%|█         | 27/260 [00:01<00:13, 17.76it/s]


 11%|█         | 29/260 [00:01<00:12, 17.84it/s]


 12%|█▏        | 31/260 [00:01<00:12, 17.83it/s]


 13%|█▎        | 33/260 [00:01<00:12, 17.84it/s]


 13%|█▎        | 35/260 [00:01<00:12, 17.82it

# Cutting a video

In [ ]:
import moviepy
t1 = 9 
t2 = 19

from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
ffmpeg_extract_subclip('/home/paula/THINKSMARTER_/videoplayback.mp4', t1, t2, targetname="test.mp4")


### CUT VIDEO AND SAVE IT

In [5]:
#cap = cv2.VideoCapture('/home/alexattia/Work/RecVis/famvk.avi')
cap = cv2.VideoCapture('/home/paula/THINKSMARTER_/videoplayback.mp4')
#cap = cv2.VideoCapture('test.mp4')
t1 = 9
t2 = 19

fps = cap.get(cv2.CAP_PROP_FPS)

initial = fps * t1
final = fps * t2

fourcc = cv2.VideoWriter_fourcc('X','V','I','D')
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

out = cv2.VideoWriter('test_2.avi',fourcc, fps , size)

count = 0

while(count < final):
    
    ret, frame = cap.read()
    if ret :
        if count > initial:
            print(count)
            #frame = cv2.resize(frame,(size[0]//3,size[1]//3))
            out.write(frame)
        count += 1
    else: 
        break
    
cap.release()
out.release()

216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455


## Gif Production without counting

In [33]:
#cap = cv2.VideoCapture('/home/alexattia/Work/RecVis/famvk.avi')
cap = cv2.VideoCapture('/home/paula/THINKSMARTER_/10km_de_course.wmv.mp4')

fps = cap.get(cv2.CAP_PROP_FPS)
initial_target = int(45 * fps) + 10
final_target = int(49 * fps) + 10
i = 0
frames = []
while(True):
    ret, frame = cap.read()
    i +=1 
    if i in range(initial_target, final_target, 1):
        frames.append(frame[:,:,::-1])
    if i == final_target:
        break

In [32]:
detections = []
for i, frame in enumerate(frames):
    with tf.Graph().as_default():
        b = tiny.evaluate(weight_file_path=weights_path, data_dir='.jpg', output_dir='', framee=frame,
                          prob_thresh=0.5, nms_thresh=0.1, lw=3, 
                          display=False, save=False, draw=False, print_=0)
    detections.append(b[0])
    time.sleep(0.5)

In [41]:
k = 0
images = []
for j, frame in enumerate(frames):
    img = frame.copy()
    for detect_ in detections[k]:
        pt1, pt2 = tuple(detect_[:2]), tuple(detect_[2:])
        cv2.rectangle(img, pt1, pt2, (255, 0, 0), 2)
    images.append(img)
    if j in range(0, 94, 2):
        k += 1
    cv2.imwrite('./output_video/frame_%05d.png' % j, img[:,:,::-1])